頻度論における不確実性とブートストラップ

In [36]:
import numpy as np
import pandas as pd
import plotly.express as px

1万戸の世帯における1年間のオンライン支出

In [37]:
browser = pd.read_csv("https://raw.githubusercontent.com/TaddyLab/BDS/cea195208c1a3d06405b6b404277e1b63eb38e6c/examples/web-browsers.csv")

In [92]:
# browser.to_csv("browser.csv", encoding="utf-8", index=False)

In [39]:
browser

,id,anychildren,broadband,hispanic,race,region,spend
0,1,0,1,0,white,MW,424
1,2,1,1,0,white,MW,2335
2,3,1,1,0,white,MW,279
3,4,0,1,0,white,MW,829
4,5,0,1,0,white,S,221
...,...,...,...,...,...,...,...
9995,9996,1,1,1,white,S,102
9996,9997,1,1,1,white,MW,5096
9997,9998,1,1,0,white,NE,883
9998,9999,0,1,0,white,NE,256


年間のオンライン支出額の標本平均、分散、標準偏差は以下のようになる

$$
\bar{x} = \frac{1}{N} \sum_{i=1}^{N}{x_i} \\
var(\bar{x}) = var(\frac{1}{N} \sum_{i=1}^{N}{x_i}) = \frac{1}{N^2} \sum_{i=1}^{N}var(x_i) = \frac{\sigma^2}{n} \\
sd(\bar{x}) = \sqrt{var(\bar{x})} = \frac{\sigma}{\sqrt{n}}
$$

In [40]:
classic_mean = browser.spend.mean()
classic_var = browser.spend.var()/1e4
classic_std = browser.spend.std()/1e2

print(f"標本平均: {classic_mean:.3f}")
print(f"標本平均の分散: {classic_var:.3f}") # 
print(f"標本平均の標準偏差: {classic_std:.3f}")

標本平均: 1946.439
標本平均の分散: 6461.925
標本平均の標準偏差: 80.386


古典統計学の大半が準拠している理論的な標本分布は、ビジネスデータサイエンスで直面する複雑な状況では成り立たない。  
そこで、ブートストラップ標本を使う。

$$
se(\hat{\beta}) \approx sd(\hat{\beta_{b}}) = \sqrt{\frac{1}{B}\sum_{b}(\hat{\beta_b} - \hat{\beta})^2}
$$

推定量が不偏である限り、この標準誤差を用いて95%信頼区間を作ることができる。  
$$
\beta \in \hat{\beta} \pm 2sd(\hat{\beta_b})
$$

In [41]:
browser.spend.sample(frac=1, replace=True).mean()

1978.0748

In [42]:
B = 10000 # ブートストラップサンプル数
mub = []
for b in range(B):
    samp_b = browser.spend.sample(frac=1, replace=True) # 復元抽出していることに注意
    mub.append(samp_b.mean())
bootstrap_mean = np.mean(mub)
bootstrap_std = np.std(mub)
print(bootstrap_std)

79.91334116600433


In [43]:
print(f"95%信頼区間: [{bootstrap_mean - 2 * bootstrap_std}, {bootstrap_mean + 2 * bootstrap_std}]")

95%信頼区間: [1785.4865313379914, 2105.1398960020088]


先ほど計算した標準偏差と一致した。

In [44]:
px.histogram(mub, height=400, width=500, nbins=50)

所感  

信頼区間とか分布に正規分布などを仮定できない場合や、標本があまりない場合、離散的な値をとる場合などに、簡単に不確実性を分析できるのがいい。  
データを元に推定値を計算する場合も、ブートストラップサンプルに対してモデルを推定して、得た推定値の分布を調べればいい。  

95%信頼区間を標準偏差×2で求めなくても、サンプルの上下2.5%の点を求めればOK  

上下2.5%の点を求めるには、1000サンプルだとちょっと心許ない。10000サンプルだと安心できる。  

ブートストラップで推定した標本分布に対する信頼性について、低次元の統計量ならいいが、2,3次元より大きい場合は再標本化に期待すべきではない。

In [45]:
# ブートストラップサンプルに基づく95%信頼区間
np.percentile(mub, [2.5, 50, 97.5])

array([1797.8219925, 1942.51175  , 2109.431095 ])

In [46]:
# 正規分布に基づく信頼区間
from scipy.stats import norm

In [47]:
norm.ppf(q=[0.025, 0.500, 0.975], loc=browser.spend.mean(), scale=browser.spend.std()/1e2)

array([1788.88543493, 1946.4393    , 2103.99316507])

全て概ね一致した。

少し複雑な例

対数支出額を回帰する  

$$
\log(spend) = \beta_0 + \beta_1 broadband + \beta_2 children
$$

In [48]:
browser.head()

,id,anychildren,broadband,hispanic,race,region,spend
0,1,0,1,0,white,MW,424
1,2,1,1,0,white,MW,2335
2,3,1,1,0,white,MW,279
3,4,0,1,0,white,MW,829
4,5,0,1,0,white,S,221


In [49]:
from statsmodels.formula.api import ols

In [50]:
formula = "np.log(spend) ~ broadband + anychildren"
model = ols(formula=formula, data=browser)
res = model.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(spend)   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     84.37
Date:                Sat, 26 Sep 2020   Prob (F-statistic):           4.63e-37
Time:                        15:37:52   Log-Likelihood:                -19223.
No. Observations:               10000   AIC:                         3.845e+04
Df Residuals:                    9997   BIC:                         3.847e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       5.6851      0.044    129.119      

In [51]:
res.bse

Intercept      0.04403
broadband      0.04357
anychildren    0.03380
dtype: float64

In [52]:
# 相関はほぼ無し
res.cov_params()

,Intercept,broadband,anychildren
Intercept,0.001939,-0.001542,-0.000653
broadband,-0.001542,0.001898,-0.000040
anychildren,-0.000653,-0.000040,0.001142


ブロードバンド接続の係数の95%信頼区間は次の通り

In [53]:
res.params.broadband + np.array([-2, 2]) * res.bse.broadband

array([0.46570632, 0.63998725])

In [54]:
tmp = np.empty((0, 3))
tmp = np.append(tmp, res.params.values.reshape(1, -1), axis=0)
tmp = np.append(tmp, res.params.values.reshape(1, -1), axis=0)
tmp

array([[5.68508452, 0.55284679, 0.08216339],
       [5.68508452, 0.55284679, 0.08216339]])

In [55]:
pd.DataFrame(res.params).T

,Intercept,broadband,anychildren
0,5.685085,0.552847,0.082163


In [56]:
# ブートストラップ推定値を求める
B = 1000
betas = pd.DataFrame(columns=["Intercept", "broadband", "anychildren"])
for b in range(B):
    samp_b = browser.sample(frac=1, replace=True)
    res = ols(formula=formula, data=samp_b).fit()
    betas = betas.append(pd.DataFrame(res.params).T, ignore_index=True)

In [57]:
betas.cov()

,Intercept,broadband,anychildren
Intercept,0.001771,-0.001409,-0.000570
broadband,-0.001409,0.001792,-0.000068
anychildren,-0.000570,-0.000068,0.000996


In [58]:
broadband_anychildren_cov = betas.cov().loc["broadband", "anychildren"]
print(f"braodbandとanychildrenの相関は{broadband_anychildren_cov:.4f}")

braodbandとanychildrenの相関は-0.0001


In [59]:
# broadbandの回帰係数のブートストラップ標本分布
px.histogram(betas, x="broadband", height=400, width=500, nbins=20)

In [60]:
# 乗法的な増加効果, ブートストラップ標本を変換するだけでOK
px.histogram(np.exp(betas), x="broadband", height=400, width=500, nbins=20)

説明変数に指数を噛ませるとオッズになるのなんでだっけ...  
ブロードバンド接続がある世帯はオンライン支出が60%から90%増加する

ブートストラップがうまくいかない場合、パラメトリックブートストラップを用いることもできる。

In [61]:
xbar = np.mean(browser.spend)
sig2 = np.var(browser.spend)

B = 1000
mub = []
for b in range(B):
    xsamp = np.random.normal(loc=xbar, scale=np.sqrt(sig2), size=10000)
    mub.append(np.mean(xsamp))

In [62]:
np.mean(mub), np.std(mub)

(1952.8642578145852, 77.92668542366704)

In [63]:
classic_mean, classic_std

(1946.4393, 80.38610214890022)

パラメトリックブートストラップでは、データが正規分布に従うという大きな仮定を行う。  
完全な確率モデルがあるため、母集団分布を知るのにそれほど多くのデータを必要とせず、高次元のパラメータでも推定できる。

不偏でない推定量にブートストラップを用いる場合、若干複雑なブートストラップアルゴリズムを用いて信頼区間を直接扱うことができる。  
例えば、標本分散は母分散の偏った推定量。

In [64]:
smallsamp = browser.spend.sample(n=100)
s = np.std(smallsamp)
s / np.std(browser.spend)

0.3578859862706105

めちゃ小さい。  
100サンプルの中に消費額が大きいユーザーがきちんと含まれていないと、標準偏差を著しく過小評価してしまう。

In [65]:
eb = []
for b in range(B):
    sb = np.std(smallsamp.sample(100, replace=True))
    eb.append(sb - s)
np.mean(eb)

-75.56695873130326

In [66]:
s

2876.7620961073576

In [67]:
np.mean(s - eb)

2952.329054838661

In [68]:
np.std(browser.spend)

8038.208274330513

In [69]:
# 90%信頼区間, 真値が含まれていることがわかる
np.quantile(s - eb, [0.05, 0.95])

array([1952.14382173, 3932.26394593])

# 仮説検定と偽発見率の制御

例えば回帰分析で有意に説明力がある説明変数を探す場合、多重検定によりp値と信頼性の関係は崩れる。  
有意水準を5%にしても、100個の係数のうち5個が実際に関係がある場合、95個の中から5%の最もらしいゴミを採用してしまう。

偽発見割合(FDP) = 偽発見率/有意な検定回数

BH法によるFDR制御  
1. p値を昇順に並べる  
2. p値が傾きq/Nの直線を上回るかを調べる  
3. p値が上回った中で最も順位が低いp値を閾値にする  

注意点  
棄却値qには0.1がよく使用される  
偽発見割合をコントロールするのが目的なので、通常の仮説検定におけるαの設定は行わない。

In [72]:
browser.head()

,id,anychildren,broadband,hispanic,race,region,spend
0,1,0,1,0,white,MW,424
1,2,1,1,0,white,MW,2335
2,3,1,1,0,white,MW,279
3,4,0,1,0,white,MW,829
4,5,0,1,0,white,S,221


In [76]:
spendy = ols(formula="np.log(spend) ~ anychildren + broadband + hispanic + race + pd.Categorical(region)", data=browser).fit()


In [77]:
print(spendy.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(spend)   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     25.58
Date:                Sat, 26 Sep 2020   Prob (F-statistic):           5.14e-44
Time:                        15:41:10   Log-Likelihood:                -19193.
No. Observations:               10000   AIC:                         3.841e+04
Df Residuals:                    9990   BIC:                         3.848e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept       

In [88]:
spendy.pvalues[1:].sort_values()

broadband                       1.323047e-32
pd.Categorical(region)[T.NE]    6.440516e-07
hispanic                        1.696801e-05
pd.Categorical(region)[T.W]     5.322114e-04
anychildren                     1.122431e-02
race[T.black]                   1.594329e-01
race[T.white]                   1.740374e-01
race[T.other]                   1.878053e-01
pd.Categorical(region)[T.S]     8.977425e-01
dtype: float64

In [89]:
q = 0.1
N = 9
fdr = pd.DataFrame({
    "order": list(range(1, 10)),
    "pvalue": spendy.pvalues[1:].sort_values(),
    "bh": [q/N * i for i in range(N)]
})

In [90]:
import plotly.graph_objects as go

trace1 = go.Scatter(
    x=fdr.order,
    y=fdr.pvalue,
    mode="markers",

)
trace2 = go.Scatter(
    x=fdr.order,
    y=fdr.bh,
    mode="lines"
)
fig = go.Figure(data=[trace1, trace2])
fig.show()

上の例だと、直線を下回る最大は5番目なので、5つの説明変数を採用する  
採用した5つの説明変数のうち、誤って採用してしまった確率は10%

まとめ  

実世界は雑然としている。限られた標本から計算した推定量には誤差が含まれている。  
シグナルとノイズを区別するため、推定量の不確実性を定量化することは重要。  
(ノンパラメトリック)ブートストラップ標本を用いると推定値の分布を求められる。推定値の信頼区間を計算することができる。  
高次元のパラメータに対しては、パラメトリックブートストラップを用いる必要がある。  

仮説検定について、検定を繰り返す場合、p値に基づく信頼性の判断は偽陽性を高めてしまう恐れがある。  
BH法を用いて偽発見割合(FDR)を直接制御することで、偽陽性を抑えつつ関連度の高い説明変数を抽出できる。